In [1]:
#default_exp segmentation.metrics

In [2]:
%autosave 60 
import fastai; print(fastai.__version__)

Autosaving every 60 seconds
1.0.58.dev0


In [3]:
#export
from fastai.vision import *
from fastai.metrics import foreground_acc, dice

In [4]:
#export
_all_ = ["foreground_acc", "dice"]

### test

In [5]:
from local.test import *

In [6]:
a = tensor([[
    [
    [0,0,1],
    [0,0,1],
    [1,1,1]
],
    [
    [1,1,0],
    [1,1,0],
    [0,0,0]
]
]])

b = tensor([[[
    [0,0,0],
    [0,1,1],
    [0,1,1]
]]])

a.size(), b.size()

(torch.Size([1, 2, 3, 3]), torch.Size([1, 1, 3, 3]))

In [7]:
test_eq(dice(a,b, eps=1e-8), (2*1)/(8))

### `iou`

In [8]:
#export
def iou(input: torch.Tensor, targs: torch.Tensor, **kwargs)->Rank0Tensor:
    "Binary IOU"
    return dice(input, targs, iou=True, **kwargs)

In [9]:
test_eq(iou(a,b, eps=1e-8), 1/7)

### `multilabel_dice`

In [65]:
#export
def multilabel_dice(input:Tensor, targs:Tensor, c:int, iou:bool=False, 
                    mean=True, eps:float=1e-8, sigmoid:bool=False, threshold:float=0.5)->Rank0Tensor:
    "Batch/Dataset Mean Dice"
    if sigmoid:
        sigmoid_input = input.sigmoid()
        thresholded_input = sigmoid_input > threshold
        _, indices = torch.max(sigmoid_input, dim=1)
        values, _ = torch.max(thresholded_input, dim=1)
        input = (values.float()*indices.float()).view(-1)
    else:
        input = input.argmax(dim=1, keepdim=True).view(-1)
        
    targs = targs.view(-1)
    res = []
    for ci in range(c):
        _input, _targs = input == ci, targs == ci
        intersect = (_input * _targs).sum().float()
        union = (_input+_targs).sum().float()
        if not iou: res.append((2. * intersect / union if union > 0 else union.new([1.]).squeeze()))
        else: res.append(intersect / (union-intersect+eps))
    res = torch.tensor(res).to(input.device)
    if not mean: return res
    else: return res.mean()

In [66]:
test_eq(multilabel_dice(a,b,2, eps=1e-8), ((2*1)/(8) + (2*2)/(10))/2)

### `multilabel_iou`

In [87]:
#export
def multilabel_iou(input: torch.Tensor, targs: torch.Tensor, c)->Rank0Tensor:
    "Batch/Dataset Mean IOU"
    return multilabel_dice(input, targs, c=c, iou=True)

In [88]:
test_eq(multilabel_iou(a,b,2), (1/7+2/8)/2)

### `sigmoid_multilabel_dice`

In [93]:
#export
def sigmoid_multilabel_dice(input, target, c, threshold): 
    return multilabel_dice(input, target, c=c, threshold=threshold, sigmoid=True)

### `sigmoid_multilabel_iou`

In [94]:
#export
def sigmoid_multilabel_iou(input, target, c, threshold):
    return multilabel_iou(input, target, c=c, threshold=threshold, sigmoid=True, iou=True)

### export

In [95]:
from local.notebook.export import notebook2script
notebook2script(all_fs=True)

Converted 00_test.ipynb.
Converted 01_script.ipynb.
Converted 02_scheduler.ipynb.
Converted 03_callbacks.ipynb.
Converted 10_segmentation_dataset.ipynb.
Converted 11_segmentation_losses_mulitlabel.ipynb.
Converted 11b_segmentation_losses_binary.ipynb.
Converted 12_segmentation_metrics.ipynb.
Converted 13_segmentation_models.ipynb.
Converted segmentation_training.ipynb.


### fin